In [1]:
from neo4j import GraphDatabase, basic_auth

import json
import gensim
import numpy as np
from collections import Counter
from itertools import combinations
from sklearn.metrics.pairwise import cosine_similarity
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

Базовый лексикон - это словарь Натальи Лукашевич, Николая Русначенко и др.

Он насчитывает более 6000 предикатов, объединенных в 277 тональных фреймов на основе предположения о существовании набора отношений между аргументами предиката (предикаты внутри одного фрейма разделяют один и тот же набор отношений между аргументами).

Почему фреймы тональные?
Потому что каждое отношение является позитивным или негативным. Вообще эти фреймы потом гипотетически используются для извлечения отношений между сущностями из текста.

In [2]:
# загружаем базовый лексикон
with open ("collection.json", encoding="utf-8") as f:
    base_lexicon = json.load(f)
# вот так выглядит типичный тональный фрейм
base_lexicon['0_62']

{'title': ['повредить', 'испортить'],
 'variants': ['вред',
  'вредить',
  'вывих',
  'вывихивать',
  'вывихнуть',
  'губить',
  'загубить',
  'запарывать',
  'запороть',
  'изувечивать',
  'изувечить',
  'изуродовать',
  'исковеркать',
  'искорежить',
  'испортить',
  'калечить',
  'коверкание',
  'коверканье',
  'коверкать',
  'корежить',
  'навредить',
  'нанесение вреда',
  'нанесение травмы',
  'нанесение урона',
  'нанесение ущерба',
  'нанести вред',
  'нанести повреждение',
  'нанести серьезный удар',
  'нанести травму',
  'нанести урон',
  'нанести ущерб',
  'наносить вред',
  'наносить повреждение',
  'наносить серьезный удар',
  'наносить травму',
  'наносить урон',
  'наносить ущерб',
  'напортить',
  'перекалечить',
  'повредить',
  'повреждать',
  'повреждение',
  'погубить',
  'подпортить',
  'покалечить',
  'попортить',
  'портить',
  'порча',
  'принесение вреда',
  'принести вред',
  'приносить вред',
  'причинение вреда',
  'причинение урона',
  'причинение ущерба',


## Небольшая подготовка

In [3]:
variants = Counter()
for frame in base_lexicon:
    for var in base_lexicon[frame]['variants']:
        if (' ' not in var) and (('INFN' or 'VERB') in morph.parse(var)[0].tag):
            variants.update([var])
print ('Всего вариантов глаголов:', len(variants))

var_list = []
for var in variants:
    if variants[var] > 1:
        pass
#         print (var)
    else:
        var_list.append(var)
print ('Выкинули повторяющиеся варианты, осталось:', len(var_list))

Всего вариантов глаголов: 2785
Выкинули повторяющиеся варианты, осталось: 2639


In [4]:
skipgram_model = gensim.models.KeyedVectors.load_word2vec_format(
    'D:\\rusvectores\\news_mystem_skipgram_1000_20_2015.bin.gz', binary=True)

# get embedding
# skipgram_model[verb + '_V']

In [5]:
# проще нагенерить пар и создавать связи между парами только если их нет
skipgram_var_list = [var for var in var_list if var+'_V' in skipgram_model.vocab]
print ('Глаголов в словаре модели:', len(skipgram_var_list))

# relations_count.value()[0]

Глаголов в словаре модели: 1528


In [6]:
%%time
# сначала наделаем эмбеддингов

def make_emb_dict(skipgram_var_list):
    emb_dict = {}
    for var in skipgram_var_list:
        emb = np.atleast_2d(skipgram_model[var + '_V'])
        emb_dict.update({var:emb})
    print ('Эмбеддингов в словаре:', len(emb_dict))
    return emb_dict

emb_dict = make_emb_dict(skipgram_var_list)

Эмбеддингов в словаре: 1528
Wall time: 15 ms


In [7]:
%%time

# в верхнетреугольную матрицу положим близость
skipgram_matrix = np.zeros(shape=(len(skipgram_var_list),len(skipgram_var_list)))

for i, verb1 in enumerate (skipgram_var_list):
    emb1 = emb_dict[verb1]

    for j, verb2 in enumerate (skipgram_var_list):
        if i < j:
            emb2 = emb_dict[verb2]
            skipgram_matrix[i, j] = cosine_similarity(emb1, emb2)

Wall time: 6min 13s


In [8]:
print (str(skipgram_matrix[1526, 1527]))
skipgram_matrix[1527, 1526] == skipgram_matrix[1526, 1527]

0.5354784727096558


False

## CRUD, Sort, Label propagation algorithm

In [78]:
driver = GraphDatabase.driver(uri="bolt://localhost:7687", 
                              auth=basic_auth("neo4j", "password"),
                             encrypted=False)
session = driver.session()

Будем класть в базу по одному фрейму

In [29]:
# CREATE

def create_frame(frame, filter_with, base_lexicon=base_lexicon):
    '''Parameters:
    
    frame: str'''
    query_count = 0
#     складываем уникальные глаголы
    for variant in base_lexicon[frame]['variants']:
#         здесь нужен список уникальных глаголов
        if variant in filter_with:
            cypher_query = "create (p:Predicate {lemma: '"+variant+"' , pos:'verb', frame: '"+frame+"'})"
            session.run(cypher_query, parameters={})
            query_count +=1
    
#     складываем аргументы
    for argument in base_lexicon[frame]['roles'].keys():
        description = base_lexicon[frame]['roles'][argument]
        cypher_query = "create (a:Argument:"+argument+" {role: '"+description+"', frame: '"+frame+"'})"
        session.run(cypher_query, parameters={})
        query_count +=1

    try: # слот polarity есть не в каждом фрейме, так что пытаемся
#         пытаемся сложить отношения
        for pol in base_lexicon[frame]['frames']['polarity']:
            
            if pol[0] == 'author':
#                 автор добавляется, коль скоро в нем есть потребность
                cypher_query = "merge (a:Argument:"+pol[0]+" {frame: '"+frame+"'})"
                session.run(cypher_query, parameters={})
                query_count +=1

            q1 = "match (x:Argument:"+pol[0]+" {frame:'"+frame+"'}), (y:Argument:"+pol[1]+" {frame:'"+frame+"'})"
            q2 = "merge (x)-[:HAS_ATTITUDE {polarity:'"+pol[2]+"', confidence:"+str(pol[3])+", frame:'"+frame+"'}]->(y)"
            cypher_query = q1+' '+q2
            session.run(cypher_query, parameters={})
            query_count +=1
            
    except KeyError:
        pass

    print ('Number of queries:', query_count)

In [69]:
%%time
# CREATE
create_frame('0_0', skipgram_var_list)

Number of queries: 9
Wall time: 37 ms


In [52]:
# READ
# посмотрим, какие глаголы из фрейма легли в базу
def read_verbs(frame=False, verbose=True):
    if frame:
        cypher_query = "match (p:Predicate) where p.pos = 'verb' \
        and p.frame = '"+frame+"' return p.lemma"
    else:
        cypher_query = "match (p:Predicate) where p.pos = 'verb' \
        return p.lemma"
        
    results = session.run(cypher_query)
    
    if verbose:
        for res in results.value():
            print (res)
    else:
        return results

In [70]:
%%time
# READ
read_verbs('0_0', True)

чаять
уповать
понадеяться
надеяться
рассчитывать
ждать
Wall time: 33 ms


In [71]:
%%time
create_frame('0_1', skipgram_var_list)
read_verbs('0_1', True)

Number of queries: 5
доверяться
полагаться
Wall time: 345 ms


In [72]:
create_frame('0_2', skipgram_var_list)
read_verbs('0_2', True)

Number of queries: 14
задыхаться
отравляться
почить
скончаться
умирать
вымирать


In [ ]:
# UPDATE
# добавим seed_label
# DELETE
# удалим seed_label

Мы положили три фрейма. "Экспертно" выберем два сидовых глагола, от которых будем распространять лейблы.

In [43]:
# UPDATE

def set_seed_labels(seed_verbs, seed_labels, verbose=True):
    '''Parameters:
    
    seed_verbs: list
    
    seed_labels: list'''
    assert len(seed_verbs) == len(seed_labels)
    seed_dict = dict(zip(seed_verbs, seed_labels))
    for verb in seed_dict:
        cypher_query = "MATCH (n:Predicate) \
        WHERE n.lemma = '"+verb+"' AND NOT (EXISTS (n.seed_label)) \
        SET n.seed_label = "+str(seed_dict[verb])
        session.run(cypher_query)
        if verbose:
            print ('Глагол: '+verb+', лейбл: '+str(seed_dict[verb]))

In [73]:
# UPDATE
seed_verbs = 'надеяться умирать'.split()
seed_labels = [1, 2]
set_seed_labels(seed_verbs, seed_labels)

Глагол: надеяться, лейбл: 1
Глагол: умирать, лейбл: 2


In [146]:
# DELETE
# потрем ненужные лейблы
def delete_seed_labels(labels=False):
    '''Parameters:
    
    labels: list of labels or False to delete all seed labels
    '''
    if labels: 
        for lab in labels:
            cypher_query = "match (n) \
            where n.seed_label = "+str(lab)+" \
            remove n.seed_label"
            session.run(cypher_query)
    else:
        cypher_query = "MATCH (n) \
        where n.seed_label is not null \
        remove n.seed_label"
        session.run(cypher_query)

def delete_frame(frame):
    cypher_query = "MATCH (n) where n.frame = '"+frame+"' DETACH DELETE n"
    results = session.run(cypher_query)
    for res in results.values():
        print (res)
    
def delete_all():
    cypher_query = "MATCH (n) DETACH DELETE n"
    results = session.run(cypher_query)
    for res in results.values():
        print (res)

In [50]:
%%time
delete_seed_labels([1,2])

Wall time: 2 ms


Добавим отношения близости. Используем загруженную выше модель. Отношения будем добавлять только один раз.

In [65]:
# CREATE
def relate_nodes(skipgram_model):

    query_count = 0

    # это лист лежащих в базе глаголов
    list2combine = read_verbs(frame=False, verbose=False).value()
    # это все возможные пары глаголов
    pairs = list(combinations(list2combine, 2))

    for verb1, verb2 in pairs:

        emb1 = np.atleast_2d(skipgram_model[verb1 + '_V'])
        emb2 = np.atleast_2d(skipgram_model[verb2 + '_V'])
        cos_sim = str(float(cosine_similarity(emb1, emb2)))

        cypher_query = "match \
        (verb1:Predicate {lemma:'"+verb1+"'}), \
        (verb2:Predicate {lemma:'"+verb2+"'}) \
        merge (verb1)-[:SIMILAR_TO {skipgram_similarity:"+cos_sim+"}]->(verb2)"
#             merge (verb2)-[:SIMILAR_TO {skipgram_similarity:"+cos_sim+"}]->(verb1) \

        session.run(cypher_query)
        query_count += 1
    print ('Запросов в базу:', query_county_count)

In [75]:
%%time
relate_nodes(skipgram_model)

Запросов в базу: 91
Wall time: 3.53 s


Попробуем присвоить метки другим глаголам на основании семантической близости (label propagation algorithm). Будем использовать ненаправленные отношения с весом, равным косинусной близости.

In [138]:
def run_lpa(relationshipProperties, maxIterations=10, verbose=True):
    
    cypher_query = '''CALL gds.graph.create(
    'myGraph',
    'Predicate',
    'SIMILAR_TO',
    {
        nodeProperties: 'seed_label',
        relationshipProperties: "'''+relationshipProperties+'''"
    })
    '''
    session.run(cypher_query)
    
    cypher_query = '''
    CALL gds.labelPropagation.stream('myGraph', 
    {maxIterations:'''+str(maxIterations)+''', 
    relationshipWeightProperty: 'skipgram_similarity', 
    seedProperty: 'seed_label'})
    YIELD nodeId, communityId AS Community
    RETURN gds.util.asNode(nodeId).lemma AS Lemma, Community
    ORDER BY Community, Lemma
    '''
    lpa_results = session.run(cypher_query)
    
    try:
        cypher_query = '''CALL gds.graph.drop('myGraph') YIELD graphName'''
        session.run(cypher_query)
    except:
        pass
    if verbose:
        for lemma, community in lpa_results.values():
            print(lemma, '\t\t', community)
    else:
        return lpa_results

In [139]:
%%time
run_lpa('skipgram_similarity', 1, True)

понадеяться 		 1
отравляться 		 2
почить 		 2
скончаться 		 2
задыхаться 		 4
умирать 		 43
чаять 		 44
уповать 		 45
надеяться 		 47
доверяться 		 63
вымирать 		 68
ждать 		 68
полагаться 		 68
рассчитывать 		 68
Wall time: 23 ms


In [140]:
%%time
# на 10 итерациях уже одно сплошное коммьюнити
run_lpa('skipgram_similarity', 10)

вымирать 		 68
доверяться 		 68
ждать 		 68
задыхаться 		 68
надеяться 		 68
отравляться 		 68
полагаться 		 68
понадеяться 		 68
почить 		 68
рассчитывать 		 68
скончаться 		 68
умирать 		 68
уповать 		 68
чаять 		 68
Wall time: 33 ms


Ничего не получилось. Предполагаемые причины:
1. Веса связей слишком незначительно отличаются.
2. Несбалансированные классы, (в смысле, коммьюнити).
3. Мало узлов с предписанными коммьюнити.

Дальнейший план действий:
* Подбирать итерации на графах с большИм (и бОльшим) количеством узлов с предписанными коммьюнити.
* Что-то делать с весами.
* Возможно, добавить отношений, используя другую модель.

## Всё заново. Возьмем другие фреймы

In [159]:
delete_all()

In [160]:
%%time
# CREATE
create_frame('0_2', skipgram_var_list)
create_frame('0_30', skipgram_var_list)


Number of queries: 14
Number of queries: 22
Wall time: 1.07 s


In [161]:
read_verbs(frame=False, verbose=True)

аплодировать
благословлять
возвеличивать
восхвалять
зааплодировать
захлопать
нахваливать
поаплодировать
поощрять
похвалить
рассыпаться
рукоплескать
славить
хвалить
хлопать
вымирать
задыхаться
отравляться
почить
скончаться
умирать


In [162]:
# UPDATE
seed_verbs = '''почить
скончаться
умирать
поощрять
похвалить
благословлять
'''.split()

seed_labels = [1,1,1,2,2,2]
set_seed_labels(seed_verbs, seed_labels)

Глагол: почить, лейбл: 1
Глагол: скончаться, лейбл: 1
Глагол: умирать, лейбл: 1
Глагол: поощрять, лейбл: 2
Глагол: похвалить, лейбл: 2
Глагол: благословлять, лейбл: 2


In [163]:
%%time
relate_nodes(skipgram_model)

Запросов в базу: 210
Wall time: 7.46 s


In [164]:
%%time
run_lpa('skipgram_similarity', 10, True)

аплодировать 		 1
благословлять 		 1
возвеличивать 		 1
восхвалять 		 1
вымирать 		 1
зааплодировать 		 1
задыхаться 		 1
захлопать 		 1
нахваливать 		 1
отравляться 		 1
поаплодировать 		 1
поощрять 		 1
похвалить 		 1
почить 		 1
рассыпаться 		 1
рукоплескать 		 1
скончаться 		 1
славить 		 1
умирать 		 1
хвалить 		 1
хлопать 		 1
Wall time: 42 ms


In [168]:
%%time
run_lpa('skipgram_similarity', 1, True)

благословлять 		 1
вымирать 		 1
задыхаться 		 1
отравляться 		 1
почить 		 1
рассыпаться 		 1
рукоплескать 		 1
скончаться 		 1
славить 		 1
умирать 		 1
хвалить 		 1
хлопать 		 1
аплодировать 		 2
возвеличивать 		 2
восхвалять 		 2
нахваливать 		 2
поаплодировать 		 2
зааплодировать 		 104
поощрять 		 112
похвалить 		 112
захлопать 		 113
Wall time: 35 ms


Резюме: всё плохо (пока). Не опускаем руки!